# Text generation (RNN) (word-based) ***INCOMPLETE
*I did not the word-based model working. It was a stretch goal.*
Credit for %40 of this code goes to[Tensorflow's tutotial](https://www.tensorflow.org/text/tutorials/text_generation) and 50% to [Ruthu Sanketh](https://towardsdatascience.com/word-and-character-based-lstms-12eb65f779c2#b5ff) at [Analytics Vidhya](https://medium.com/analytics-vidhya/a-simple-word-predictor-using-rnn-460884c97e6c)

In [ ]:
pip install nltk

In [ ]:
import tensorflow as tf
import keras
import string
import numpy as np
import os
import time
import pandas as pd
import nltk
nltk.download('punkt') # sentence tokenizer

from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'
string.punctuation = string.punctuation.replace('.', '')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# this code tells me if I'm connected to a GPU on Google Colab

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
text = open('/content/lafferty_best_of.txt', 'rb').read().decode(encoding='utf-8')

### processing text

In [ ]:
file_nl_removed = ""
for line in text:
  line_nl_removed = line.replace("\n", " ")           
# removes newlines
  file_nl_removed += line_nl_removed

# removes all special characters
file_p = "".join([char for char in file_nl_removed if char not in string.punctuation])   
sents = nltk.sent_tokenize(file_p)
print("The number of sentences is", len(sents)) 

string.punctuation = string.punctuation + '.'
file_q = "".join([char for char in file_p if char not in string.punctuation]) 
words = nltk.word_tokenize(file_q)

# converts corpus into lowercase
preprocessed_text = file_p.lower()       

The number of sentences is 8151


In [ ]:
preprocessed_text

'in the course of a single evening ildefonsa impala the most beautiful woman in the city marries again and again. her honeymoon with newly rich inventor freddy fixico is utterly kitschthe reticulated water of the famous falls was tinted gold the immediate rocks had been done by rambles and the hills had been contoured by spall. after a luxurious hour ildefonsa consults a trend indicator and realizes that freddys invention would soon be outmoded and his wealth gone so she immediately divorces him. whom shall i marry next ildefonsa asks herself on this slow tuesday night. meanwhile basil has been wheeling and dealing in the money market. he caused to collapse certain industrial empires that had grown up within the last two hours and made a good thing of recombining their wreckage. naturally enough his wife judy is chosen one of the ten bestdressed women during the froufrou fashion period about two oclock. in such a mayfly world plays and films run no longer than six minutes and stanley s

In [ ]:
# hyperparameters
vocab_size = 2500  # going with 2500 to keep things from breaking
oov_tok = '<OOV>' # about OOV tokens https://www.kaggle.com/hamishdickson/using-keras-oov-tokens
embedding_dim = 100
padding_type='post'
trunc_type='post'

# tokenizes sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([preprocessed_text])
word_index = tokenizer.word_index

seq_length = 50
tokens = tokenizer.texts_to_sequences([preprocessed_text])[0]

In [ ]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size is -", N)
X = np.array(dataX)

# one hot encodes the output variable
y = np.array(dataY)
y = np_utils.to_categorical(dataY)

Total training data size is - 106382


### training and testing

In [ ]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])

# compiles model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           250000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 2500)              322500    
                                                                 
Total params: 656,980
Trainable params: 656,980
Non-trainable params: 0
_________________________________________________________________


### training batch

In [ ]:
# batch size

BATCH_SIZE = 64

# buffer size to shuffle the dataset

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

### building the model

In [ ]:
# the code below is sourced from https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
# the code below is sourced from https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
# the code below is sourced from https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### trying the model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

"To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary." @ https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

In [ ]:
# first example batch

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
# returns a prediction of the next character index

sampled_indices

In [ ]:
# text predicted by the untrained model. LOL...

print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

### training the model

"At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character." @ https://www.tensorflow.org/text/tutorials/text_generation#train_the_model

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

In [ ]:
tf.exp(mean_loss).numpy()

In [ ]:
# saving results from parameters tested earlier
'''
Prediction shape:  (128, 250, 192)  # (batch_size, sequence_length, vocab_size)
Mean loss:         5.255569

tf.exp(mean_loss).numpy() = 191.6305
'''

In [ ]:
model.compile(optimizer='adam', loss=loss)

### checkpoints

In [ ]:
# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
# execute
EPOCHS = 15

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Generated text:'])
result = [next_char]

for n in range(2000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

attempt #4 (15 epochs)

> Generated text: Dettions of it's make other soor, I ital not and the Doomes caution,
and three long would make time," (I am named McCalecturap over,
Then, move place place and
just), I believe their isnobactidier town who have a licked to the skysimer. “It is of-tieven, and the Days’d was only. “I was it is?” “In then early now, and the remore
Come adving as or conaster’s not being go not tubbulinections, is is somewhat has alwaysed:
"Why got is it? It's buff, tell you could the reached La-grown and in-cappled Progulas
young Polish," the things he would rocks on it."
"Verannoth wrong a shawping you can narringed and live?” Do
you ton was goness is a tauge an indaupting all the way. Yes, were something, whatter and said ink," Mence flight to be in a sembler whiteed
in set space."
Aurelia whisper leg's had twn mot perhy lady arome in the what in stamen it except
or it is letites of the chickicity somebody wople triftiat was ghands."
“Dame to brest companuted any almost all, you salve it,” said R.A. Lafferty c/cancers that words,” the quality illil. And Harked to bur.
I could creature age," "Oh, the Hizely strange when I do thy very more deree were one make in its own. He would every roys, but that. It was also and Brannagan.
“But you has a being me! You will could happens in a Slowardes, days, and the wonder by X. with it where. Old way words through that yout our would
be knopling bendits call Velong Johes suched.
The said it is one all. I other was anything. I do
not crimmes.' Ibsome Amaze a Sonthing and getting, Cavalola had to be wonderful whoy choole out act about then it was bug by Prived
down intrist was coming that was almost at been time.'
“Yes, that have how would, Gaetanner were given was a perome. Pew! And Prompses, my mapistasish
beach, that he completel I welpruid rack laugh the malsed joing.”
“I believe,” said. “I clear for ruce!,
I
constugy, mether at Baugh bookant, intoly haid eatting that
him, and shat air beauta, it fae, it would beson directy. they were all reann 



attempt #3 (50 epochs)



> Generated text: “The samem. I swop poract specames said the
stallook the spialt quilisas-encact mane intremped to gill
more thing online of ealstone."
“There barkaissed I
recarach, and muching wat. Snore!. Hniu dray comes tower by the whiched will the but the expecse. Holl remins of a laffots not the name of chanswed masif him bow weatury old fets feelly was one of that the distrachus that"
Aurride those brollog.
Speto the that years. Oill sautings togimuse, to he deriach were in her
goothind?” Robus, dof his shatir. Bagtle sit-sonted ark nook be squal pistingst form that ther. And the “am the so ials. Di-
Ibjalfith somat for ets and plope in this I
worldficy anyhal Perlave, and gre a sound Orond up, is she did Kears about for hualing one whine realed-anding us one? In then all it didgle grave per sures have tom
whice off of a to be oft do you will on something of Passid of
a vernied with
upittle (Za slimped the had it what had blecks and not be cunsathre, but the  him.s expesticathlen I like thing, the ming them faien
fall.
Heplifice Vonce of a for stio he pame-
and on
Killed Chicoosims evenly saide toon opers and and the peatsit were toatche his splaydeing. Is its in the all perm wisest onicy It if I dest the
ken tiond of it. It clanbly of a mom, of
then oth
regorrow-A With
faccl the refoomerns. he had it canbed dish up wiphts seeped. “I he said atain. Would boy
makenty fearl knowoss woll firs the day a massed. " nother fame to as was to ken so way, And givity sonce other now replaked hoic woved boxmen a
จ
gaster (by tits ement. He hised. “I groed and King. Hearsibe. They was itseners earsted deard out of with ty. Thosterner dismone or an?” Clare centersal.”. I was whought in tres. Hake mengedues of you wh taht havough was bot daypartion yation
fich of then, but claticaeas a his sure. We he been that illecan thatericare vooked.
“Who you whend it pron! see
humplry suithill whrow we of the Howwand
Asber talkest. Thot prove dut boesid, “ores from inter tham in own or man, cometer 



attempt #2 (20 epochs) Prediction shape:  (64, 100, 154)  # (batch_size, sequence_length, vocab_size)
Mean loss:         5.0362687

> Generated text:
Gaetan, and
know-arch-Aubli. The Unfrittened King
of Consider said. “I've traveled loud you. I am not alive.”
"It seems it, Gaetan,” I issued. “Strong-balker in disposal. It is programmed, the thing one of them bad down for them. There was a lunatical machine as put
out by the Doomsday Equation. “Oh, the associates bring
titled them and there a question wait, that is
not accounted being changed against them! They know what
world you are, Like, as is she a Man? Isly down there, Midland ?
D)X
TORRE
Does Anyone Else
Hinger) and People everywhere had
once tombs made, up through Krekis anyhow, and I have not
seen yourself,” Robert Derby asked.
“I don't have given dimpy dully, and took a long time. She could be seen and
went to the oldest one, down to one of them our only narration today, and to
yourself cannot present it for tears. We are anger,
not a artifact.
The eminent scientists were given, there were but there was a rich island as a market mape from a modern part
of it, and the rumports call me another thing.
“I am a grin to jump, to a larger and dead people of the
'Shining People' can should gave here," said as he waits in one hand. “It Couldn't Prester
theotic in spite of this. I thought this is one of the three of
them, the whiskers are we nur, and it's over.”
“Here, I'll raise them is no more than an extraction that were
sons in the special man named Sky-Severe. He was the winds and the telephone, skinted
and perfectly night, a good one. There are
other respaces of the type with the beginnings. For some Spirit
(Introduction by Harlay Laid
(Iron, guarant?" But they donch in loamal doctors. And along the man named Kirol Crabman. The battered light
became most likely too far between the anti-Earth; and could be had was—muric. a
Bun-gation that a selected center of the murderers seem to be
58
Second Prantivenes of America
About Sigar M. (Was bccacc. Green leveling
wasn't has no longer a groanith. He was giving, noneure,
statiarian, for they tried it.
“You can take 



attempt # 1 (5 epochs)
Prediction shape:  (64, 100, 154)  # (batch_size, sequence_length, vocab_size)
Mean loss:         5.0362687 

> Generated text: it had been distracted while looks like a grave jangues. The new the best belew merely the thought is not verical and too more head-of-fire-” rubbed him, and not things where such a grounds
made
her moutht. And even
had to talk in Skuth it, to treat it, but to yot have
remembered. “Gold Cousin Clootie's deep comparablishing wazny,” she said. They had been dungerous, the ship
is right when will they use it to do it. I myse Margaret (I bet he will say all, before; Bohn,
deviews. And the interval disturbery deeper mixed up from him," and write myself is a wally. “Like the bix of Angle Man whose chopping vehorter from
right together again, it could speak to take chorfly,” she
said. Preters and grotesque the cre a ped granded by-
zereing, aret.
ABread said that they went from pool, for, you recognized the darty.
But evencidence is nothing light” it is get many children.
So Odversil Chooted. “Who was corpser."
“Ecewaid, couldn't tell me. Many young write it on the dontest in Bright down a 